In [25]:
import nfldb
import json
import pandas as pd
import datetime

In [2]:
db = nfldb.connect()

In [3]:
q = nfldb.Query(db).game(season_year='2015', season_type='Postseason')

In [40]:
team_file = 'playoff_2016_team_master.json'
with open(team_file) as f:    
    teams = json.load(f)

In [19]:
point_file = 'playoff_2016_point_key.json'
with open(point_file) as f:    
    point_key = json.load(f)

In [6]:
def player_stats(db, player_args, game_args={}):
    # find potential list of players
    players = nfldb.Query(db).player(**player_args).as_players()
    # get list of player ids
    ids = [getattr(pp,'player_id') for pp in players]
    
    player_data = []
    for player in players:
        pid = getattr(player, 'player_id')
        stats = nfldb.Query(db).game(**game_args).player(player_id=pid).as_aggregate()
        if len(stats) == 0:
            stats = None
        elif len(stats) == 1:
            stats = stats[0]
        else:
            print('something wrong with stats')
        player_data += [{'info':player, 'stats':stats}]
    
    return(player_data)

In [7]:
def player_points(player_stats, point_key):
    rows = []
    
    for player in player_stats:
        stat_key = point_key.keys()
        
        if player['stats'] is None:
            # all stats are 0 if none found
            row = {stat: 0 for stat in stat_key}
        else:
            # otherwise get stats
            row = {stat: getattr(player['stats'], stat) for stat in stat_key}
        
        row['fantasy'] = sum([row[stat] * point_key[stat] for stat in stat_key])
        row['full_name'] = player['info'].full_name
        row['team'] = player['info'].team
        rows += [row]
        
    return(rows)

In [8]:
def team_results(teams):
    results = {}
    for team in teams.keys():
        results[team] = []
        for p in teams[team]:
            ps = player_stats(db, player_args=p, game_args={'season_year':2015, 'season_type':"Postseason"})
            results[team] += player_points(ps, point_key)
    return(results)


In [9]:
def total_points(results):
    return({team_name:sum([player['fantasy'] for player in team]) for team_name, team in results.iteritems()})

In [10]:
def add_points_to_teams(teams):
    results = team_results(teams)
    for team in teams.keys():
        for i in range(len(teams[team])):
            teams[team][i].update({'points':results[team][i]['fantasy']})

In [43]:
def write_teams(teams, write_dir = 'updates_2016', write_current = True):
    if write_current:
        with open(write_dir + '/current.json', 'w') as f:
            json.dump(teams,f)
    
    time_str = datetime.datetime.now().strftime('%Y_%m_%d_%H_%M_%S')
    with open(write_dir + '/' + time_str + '.json', 'w') as f:
        json.dump(teams,f)

In [41]:
add_points_to_teams(teams)

In [44]:
write_teams(teams)